In [23]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#BiocManager::install("treeio")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")
#BiocManager::install("XVector")

#install.packages("tidyverse")
#install.packages("ggplot2")
#install.packages("ggstance")
#install.packages("ggmsa")
#install.packages("seqmagick")
#install.packages("shiny")
#install.packages("DT")
#install.packages("cowplot")
#install.packages("ggplotify")

also installing the dependency ‘gridGraphics’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [1]:
library(treeio)
library(ggtree)
library(tidyverse)
library(ggplot2)
library(ggstance)
library(Biostrings)
library(ggmsa)
library(seqmagick)
library(XVector)
library(gtable)
library(grid)
library(shiny)
library(DT)
library(cowplot)
library(ggplotify)

Registered S3 method overwritten by 'treeio':
  method     from
  root.phylo ape 

treeio v1.13.1  For help: https://yulab-smu.github.io/treedata-book/

If you use treeio in published research, please cite:

LG Wang, TTY Lam, S Xu, Z Dai, L Zhou, T Feng, P Guo, CW Dunn, BR Jones, T Bradley, H Zhu, Y Guan, Y Jiang, G Yu. treeio: an R package for phylogenetic tree input and output with richly annotated and associated data. Molecular Biology and Evolution 2020, 37(2):599-603. doi: 10.1093/molbev/msz240


ggtree v2.3.7  For help: https://yulab-smu.github.io/treedata-book/

If you use ggtree in published research, please cite the most appropriate paper(s):

- Guangchuang Yu. Using ggtree to visualize data on tree-like structures. Current Protocols in Bioinformatics, 2020, 69:e96. doi:10.1002/cpbi.96
- Guangchuang Yu, Tommy Tsan-Yuk Lam, Huachen Zhu, Yi Guan. Two methods for mapping and visualizing associated data on phylogeny using ggtree. Molecular Biology and Evolution 2018, 35(12):3041-3

## Likely, the simplest shiny app ever

In [2]:
server <- function(input, output, session) { } #the server

ui <- basicPage("This is a real Shiny app and jupyter is the best thing ever ;D") # the user interface

shinyApp(ui = ui, server = server) # this launches your app


Listening on http://127.0.0.1:7254



## Ok so let's build a web app which displays some trees
## overview: 4 parts - preprocessing, ui, server, app calling

### preprocessing

In [2]:
## define my two classes - defining the A2TEA HYPOTHESES object ;D
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df", 
                                  fasta_files="list", 
                                  msa="AAMultipleAlignment", 
                                  tree="phylo"))

# class for the hypotheses
setClass("hypothesis", slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list",
                                  species_tree="phylo"))


#load the A2TEA input
#load("/home/tyll/Desktop/PhD/A2TEA/shiny/example_trees/A2TEA_finished.RData")

### ui

In [3]:
# Define UI for tree app ----

#hypothesesList <- as.list(hypotheses$hypothesis)
#names(hypothesesList) <- hypotheses$name
#hypothesesList


# Allow specific errors to be displayed on screen, instead of displaying a generic error
options(shiny.sanitize.errors = FALSE)

ui <- pageWithSidebar(
  
  # App title ----
  headerPanel("Tree building"),
  
  # Sidebar panel for inputs ----
  sidebarPanel(
  
      fileInput(inputId = "A2TEA",
                label = "Upload A2TEA.RData file:",
                multiple = FALSE,
                accept = ".RData"),
      
        # Input: alternative way which leads to problems...
#      selectInput("select",
#                    label = h3("Select hypothesis:"),
#                    ""), 

      uiOutput('select_Hypothesis'),
      uiOutput('select_HOG')
      
  ),
  
  # Main panel for displaying outputs ----
  mainPanel(
      h3(textOutput("text")),
      plotOutput("speciesTree"),
      plotOutput("VennUpSet_plot"),
      DT::dataTableOutput("HOG_DE_DT"),
      DT::dataTableOutput("HOG_level"),
      plotOutput("recFacetPlot"),
  )
)

### server

In [4]:
#increasing maximum file size for upload!
#https://stackoverflow.com/questions/18037737/how-to-change-maximum-upload-size-exceeded-restriction-in-shiny-and-save-user
options(shiny.maxRequestSize=50*1024^2)

myDataFrame <- data.frame(names=c(""))
#rm(myDataFrame)

server <- function(input, output, session) {
    

#read-in hypotheses.tsv reactively
    tsv <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    d <- get("hypotheses", envir=t)
     }
     d
   })  
    
#    observe({
#       updateSelectInput(session, "select",
#                         label = "Select Hypothesis:",
#                         choices = tsv()$name,
#                         selected = tsv()$name[1])
# })
    
    
#    output$hypothesesList <- renderText({
#        tsv()$name
#    })
    

    
    
#    tsv2 <- reactive({
#       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs)))
#       vars <- as.list(vars)
#       return(vars)
    
    
    output$select_Hypothesis = renderUI({
      selectInput('select_Hypothesis2', 'Select Hypothesis:', tsv()$name)
    })

# also read-in the Ortho Venn/UpSet plots 
    Ortho_intersect_plots <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t5 = new.env()
    load(input$A2TEA$datapath, envir = t5)
    d <- get("Ortho_intersect_plots", envir = t5)
     }
     d
   }) 

    #plot the Ortho Venn/UpSet plots
    output$VennUpSet_plot <- renderPlot({
        Ortho_intersect_plots()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]
    })

    
# also read-in the DE genes table 
    HOG_DE <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t2 = new.env()
    load(input$A2TEA$datapath, envir = t2)
    d <- get("HOG_DE.a2tea", envir = t2)
     }
     d
   }) 


# also read-in custom A2TEA object
    HYPOTHESES <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t3 = new.env()
    load(input$A2TEA$datapath, envir = t3)
    d <- get("HYPOTHESES.a2tea", envir = t3)
     }
     d
   })  

# buld species tree
    #plot the Ortho Venn/UpSet plots
    output$speciesTree <- renderPlot({
       st <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@species_tree
       ggtree(st, layout = 'rectangular', branch.length='yes', size=1.5) +
           geom_tiplab(align=F, offset = 0.1, alpha=1)
            
    })
    
    
# also read-in hypothesis specific HOG level table - HOG_level_list
    HOG_level <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t4 = new.env()
    load(input$A2TEA$datapath, envir = t4)
    d <- get("HOG_level_list", envir = t4)
     }
     d
   })  
    
    
# normal table chokes on large size of df
# therefore usage of DT::renderDataTable
    output$HOG_DE_DT <- DT::renderDataTable({
        HOG_DE()
  })
    
    
#example access to hypothesis number - useful for access 
    hypothesisNUM <- reactive({
#       HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@number
        HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@number
    })
   
     output$text <- renderText({
        hypothesisNUM()
    })    
        
        

        
# I can make use of the hypothesis number previously created
    output$HOG_level <- DT::renderDataTable({
#        HOG_level()[[tsv()$hypothesis[tsv()$name==input$select]]]
                HOG_level()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]
  })
             
     
        


# observe choice of HOG; e.g. for building trees
# first create reactive list of all exp. HOGs 
# due to my custom object this needs the following trickery: 
# all.vars(parse(text = names(....
hypothesisExpOGs <- reactive({
#       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs)))
       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs)))    
       vars <- as.list(vars)
       return(vars)    
    })
    
# actually render the UI element - a dropdown list of the expanded HOGs of the current hypothesis
    output$select_HOG = renderUI({
      selectInput('select_HOG2', 'Select HOG', hypothesisExpOGs())
    })
    
# creating the rectangular A2TEA triple facet plot

# make renderUI element reactive
# access to element based on name in first element selectInput('')
    choice <- reactive({input$select_HOG2})
    
output$recFacetPlot <- renderPlot({

    #### building the rectangular facet plot - HOG tree, logFC & MSA
                     
#    tree <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs[[choice()]]@tree
    tree <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs[[choice()]]@tree
    info <- HOG_DE()


# extracted from colourblind - removed black... how to deal with NAs..
# also a problem if more than given colours are supposed to be displayed error....
# could perform check of number of HOGs and switch between an optimized colour scale with predefined colours
# to an open-ended/automatic colour scheme if set number is exceeded by choice of HOG
    cols <- c("#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7", "lightblue", "darkred",
              "lightblue", "darkred", "pink", "cyan", "grey", "violet", "magenta", "gold", 
              "darkgreen", "darkblue")


# rlang package has the function is_empty()
# with it we can test for "character(0)"
    el <- list()

    for (i in 1:length(tree$tip.label)) {
        if (is_empty(info$HOG[info$gene == tree$tip.label[i]]) == FALSE) {
           first <- info$HOG[info$gene == tree$tip.label[i]]
           el <- c(el, list(filler = tree$tip.label[i]))
           names(el)[i] <- first
        }
        else {
            el <- c(el, list(singleton = tree$tip.label[i]))      
        }
    }

# reduce the list to unique tags
    el_split <- sapply(unique(names(el)), function(x) unname(unlist(el[names(el)==x])), simplify=FALSE)

    p_OTU <- ggtree(tree,
                    layout = 'rectangular',
                    branch.length='none',
                    size=1.5)

    p <- groupOTU(p_OTU, el_split, 'HOG') + 
                       aes(color=HOG) +
                       geom_tippoint(aes(color=HOG)) +
                       scale_color_manual(values=cols,
                                          na.translate=TRUE,
                                          na.value = "black") +
                       geom_tiplab(aes(color=HOG), 
                                   align=F, 
                                   offset = 0.3,
                                   alpha=1)
                                   

# this is such a convenient way to do this ;D + we also drop the columns we don't need
    d <- filter(info, gene %in% tree$tip.label) %>% 
             select(-c(species, baseMean, HOG, lfcSE, stat, pvalue, padj))

    lines=data.frame(y = c(-2,-1,1,2), .panel='log2FC')

                   
# xlim set x axis limits for only Tree panel
# xlim tree calculated as number of tips tips?
    p2 <- facet_plot(p + xlim_tree(16), 
                     panel = 'log2FC', 
                     data = d, 
                     geom = geom_barh, 
                     mapping = aes(x = log2FoldChange),
                     stat='identity',
                 #inherit.aes = TRUE,
                     color='firebrick') + 
                       theme_tree2(plot.margin=margin(0, 0, 0, 0)) + 
                       geom_vline(data=lines, 
                                  aes(xintercept=y), 
                                  linetype = "dashed")
                   
                   
# get the msa and assign it to x (for now...)
# set msa range of amino acids to be displayed - make this dynamcic!
#    x <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs[[choice()]]@msa
    x <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs[[choice()]]@msa
    data <- tidy_msa(x, start = 1, end = 598)


# use font to display amino acid code, eg "helvetical"
p3 <- p2 + geom_facet(geom = geom_msa, 
               data = data,  
               panel = 'msa',
               font = NULL, 
               color = "Clustal")
                   

    gt = ggplot_gtable(ggplot_build(p3))
#gt$layout$l[grep('background', gt$layout$name)]
#gt$widths[1] = 20*gt$widths[1] # in this case it was colmun 5 - double the width
    gt$widths[5] = 2*gt$widths[5] # in this case it was colmun 5 - double the width
    gt$widths[7] = 2*gt$widths[7] # in this case it was colmun 7 - double the width
    gt$widths[9] = 5*gt$widths[9] # in this case it was colmun 9 - double the width

# this option is nice for jupyter but I have to see what will be possible in shiny                   
    options(repr.plot.width=15, repr.plot.height=8)
    grid.draw(gt)
            
            
############ ending facet plot code ############
            
  })
        
}

### app calling

In [ ]:
shinyApp(ui, server)


Listening on http://127.0.0.1:3107

Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to select less than one element in get1index”
Warning message:
“Error in .subset2: attempt to selec

### playground & working examples

In [282]:
hypotheses$hypothesis[hypotheses$name=="Expanded in barley compared to maize"]

[1] 2

In [247]:
saveRDS( data.frame(names=c("Jill","Jane","Megan")),"example_trees/myDataFrame.rds")

#### completely working mini application showcasing runtime loading of data and reactive dropdown

In [249]:
myDataFrame <- data.frame(names=c("Tom","Dick","Harry"))

ui <- shinyUI(
    fluidPage(
        fileInput("file1", "Choose file to upload", accept = ".rds"),
        selectInput("myNames","Names", ""),
        tableOutput("contents")
    )
)

server <- function(input, output, session) {

    myData <- reactive({
        inFile <- input$file1
        if (is.null(inFile)) {
            d <- myDataFrame
        } else {
            d <- readRDS(inFile$datapath)
        }
        d
    })

    output$contents <- renderTable({
        myData()
    })

    observe({
         updateSelectInput(session, "myNames",
                           label = "myNames",
                           choices = myData()$names,
                           selected = myData()$names[1])
    })

}

shinyApp(ui, server)



Listening on http://127.0.0.1:5402



#### shiny example of using RData input

In [47]:
#The example allows you to upload .RData files.
#The approach with load and get allows you to assign the loaded data to a variable name of your choice.
#For the matter of the example being "standalone" I inserted the top section that stores 
#two vectors to your disk in order to load and plot them later.


# Define two datasets and store them to disk
x <- rnorm(100)
save(x, file = "x.RData")
#rm(x)
y <- rnorm(100, mean = 2)
save(y, file = "y.RData")
#rm(y)
# both
save(x, y, file = "z.RData")

# This function, borrowed from http://www.r-bloggers.com/safe-loading-of-rdata-files/, load the Rdata into a new environment to avoid side effects
LoadToEnvironment <- function(RData, env=new.env()) {
  load(RData, env)
  return(env)
}


# Define UI
ui <- shinyUI(fluidPage(
  titlePanel(".RData File Upload Test"),
  mainPanel(
    fileInput("file", label = ""),
    actionButton(inputId="plot","Plot"),
    plotOutput("hist"))
  )
)

# Define server logic
server <- shinyServer(function(input, output) {

  observeEvent(input$plot,{
    if ( is.null(input$file)) return(NULL)
    inFile <- input$file
    file <- inFile$datapath
    # load the file into new environment and get it from there
#    name <- load(file, envir = e)
#    data <- e[[name]]
    # this works but not really what I need
#    data <- e[[name[1]]]
    t = new.env()
    load(file, envir = t)
    data <- get("y", envir=t)

      
      
#    observeEvent(input$plot,{
#        if ( is.null(input$file)) return(NULL)
#        inFile <- input$file
#        file <- inFile$datapath
#    # Use a reactiveFileReader to read the file on change, and load the content into a new environment
#    env <- reactiveFileReader(1000, session, "z.RData", LoadToEnvironment)
#    # Access the first item in the new environment, assuming that the Rdata contains only 2 items which is a data frame
#    env()[[names(env())[2]]]
    
    
    # Plot the data
    output$hist <- renderPlot({
      hist(data)
    })
  })
})

# Run the application 
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4827



In [42]:
e = new.env()
name <- load("z.RData", envir = e)
#mget(c("y","x"), envir=e)
#get returns exactly the object
test <- get("y", envir=e)
str(test)
str(y)

 num [1:100] 3.011 0.694 1.618 1.81 2.27 ...
 num [1:100] 3.011 0.694 1.618 1.81 2.27 ...


In [101]:
load("example_trees/backup_old_RData//A2TEA_finished.RData")
#str(hypotheses$name)
#HOG_DE.a2tea
#HYPOTHESES.a2tea
#HOG_level_list[[1]][HOG_level_list[[1]]$expansion=="yes",]
#tt <- as.data.frame(names(HYPOTHESES.a2tea[[1]]@expanded_OGs))
#names(tt)
#names(tt)[names(tt) == "names(HYPOTHESES.a2tea[[1]]@expanded_OGs)"] <- "HOG"
#names(tt)
#tt$HOG
#tt[[1]]
summary(HYPOTHESES.a2tea)
#?base::as.data.frame
hypotheses


load("example_trees/A2TEA_finished.RData")
#str(HOG_level_list)
#hypotheses
summary(HYPOTHESES.a2tea)


             Length Class      Mode
hypothesis_1 1      hypothesis S4  
hypothesis_2 1      hypothesis S4  

hypothesis,name,expanded_in,compared_to
<dbl>,<chr>,<chr>,<chr>
1,Expanded in Arabidopsis compared to Monocots,Arabidopsis_thaliana,Zea_mays;Hordeum_vulgare
2,Expanded in barley compared to maize,Hordeum_vulgare,Zea_mays


             Length Class      Mode
hypothesis_1 1      hypothesis S4  
hypothesis_2 1      hypothesis S4  